# Passos do Projeto

In [1]:
# criar um navegador

# importar / visualizar a base de dados

# para cada item dentro da nossa base de dados, para cada produto
    #procurar esse produto no google shopping
        #verificar se algum dos produtos do google shpiing está dentro da minha faixa de preço
    #procurar no buscape
        #verificar se algum dos produtos do buscape está dentro da minha faixa de preço
        
# salvar as ofertas boas em um dataframe (tabela)
# exportar para o excel
# enviar por emial o resultado da tabela

# Criar um navegador

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# Extra atualizar o navegador
servico = Service(ChromeDriverManager().install())
options = webdriver.ChromeOptions() 
options.add_argument("start-maximized")

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 109.0.5414
[WDM] - Get LATEST chromedriver version for 109.0.5414 google-chrome
[WDM] - Driver [C:\Users\fiu126\.wdm\drivers\chromedriver\win32\109.0.5414.74\chromedriver.exe] found in cache


# Importar e tratar as Bases de Dados

In [3]:
# Importar e tratar as Bases de Dados
import pandas as pd
from pathlib import Path
#Caminho
caminho= Path(r'C:\Users\fiu126\Desktop\Impressionador\Projeto 2 - Google Shopping e Buscape')
#print (caminho)

#importar as  base dados
tabela_produtos = pd.read_excel(r'{}\Buscas.xlsx'.format(caminho))
display(tabela_produtos)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64 gb,mini watch,300,550
1,rtx 3060,zota galax,250,400


# Definição das funções de procura do google shopping e buscape

In [4]:
import time

def verificar_tem_termos_banidos(lista_termos_banidos,nome):
    tem_termos_banidos=False
    for palavra in lista_termos_banidos:
        if palavra in nome:
            tem_termos_banidos=True
    return tem_termos_banidos

def verificar_tem_termos_produtos(lista_termos_nome_produto,nome):
    tem_todos_termos_produtos=True
    for palavra in lista_termos_nome_produto:
        if palavra not in nome:
            tem_todos_termos_produtos=False
    return tem_todos_termos_produtos
    
    

def procura_gooogle_shopping(nav,produto,termos_banidos,preco_min,preco_max):
    
    #tratar os valores
    produto=produto.lower()
    termos_banidos=termos_banidos.lower()
    lista_termos_nome_produto=produto.split(" ")
    lista_termos_banidos=termos_banidos.split(" ")
    lista_ofertas=[]
    preco_minimo=float(preco_min)
    preco_maximo=float(preco_max)

    # entrar no google
  
    nav.get("https://www.google.com/")

    #aceitar os termos -> apenas na primeira iteração é que tenho que aceitar os termos
    # se aparecer os termos tenho que aceitar para continuar
    
    if len(nav.find_elements(By.XPATH,'//*[@id="L2AGLb"]/div')) > 0:
        nav.find_element(By.XPATH, '//*[@id="L2AGLb"]/div').click()
   
    
    #popup do google -> clicar em 'Agora Não'
    if len(nav.find_elements(By.XPATH,'//*[@id="yDmH0d"]/c-wiz/div/div/c-wiz/div/div/div/div[2]/div[2]/button')) > 0:
        nav.find_element(By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div/c-wiz/div/div/div/div[2]/div[2]/button').click()
    


    #nav.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(produto) #escrever o produto
    #nav.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER) #press enter

    #codigo alternativo -> fazer em apenas uma linha
    nav.find_element(By.XPATH,
                     '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(produto,Keys.ENTER) #codigo altertativo -> escreve o nome e faz enter


    # entrar no separador do shopping (tenho que entra em 'compras' e depois 'Shopping' )
        # -> com este código garanto que o código fucniona se os itens mudarem de local -> faço uma lista de todos os itens existentes no menu (procurar a class comum) 

    elementos=nav.find_elements(By.CLASS_NAME,"MUFPAc")

    for item in elementos:
        if "Compras" in item.text:
            item.click()
            break

    elementos2=nav.find_elements(By.CLASS_NAME,"NZmxZe")

    for item in elementos2:
        if "Shopping" in item.text:
            item.click()
            break

    # retirar todos os preços -> fazer uma lista de resultados e pesquisar dentro da lista de resultados
     # -> vou querer o nome, preço e a loja para cada item 

    lista_resultados=nav.find_elements(By.CLASS_NAME,"KZmu8e")

    for resultado in lista_resultados:
        #nome=resultado.text.split("\n")[0]
        nome=resultado.find_element(By.CLASS_NAME,"ljqwrc").text.split("\n")[0]
        nome=nome.lower()

        # analisar se não tem termo banido
        tem_termos_banidos=False
        for palavra in lista_termos_banidos:
            if palavra in nome:
                tem_termos_banidos=True

        #analisar se tem todos os termos do nome do produto
        tem_todos_termos_produtos=True
        for palavra in lista_termos_nome_produto:
            if palavra not in nome:
                tem_todos_termos_produtos=False

        # selecionar só os elementos que só tem_termos_banidos=False e tem_todos_termos_produtos=True
        #if tem_termos_banidos==False and tem_todos_termos_produtos==True:

        if not tem_termos_banidos and tem_todos_termos_produtos:

            #preco=resultado.text.split("\n")[1].split(" ")[0][0:8]
            preco=resultado.find_element(By.CLASS_NAME,"T14wmb").text.split(" ")[0]
            preco=preco.replace('€','').replace(' ','').replace(',','.')
            preco=float(preco)

            
            # se o preco ta entre o preco_minimo e preco_maximo:
            if preco_minimo<=preco<=preco_maximo:
                # caso especial -> não consigo selecionar o link  -> vou indiretamente
                elemento_referencia=resultado.find_element(By.CLASS_NAME,"HUOptb") #classe do filho
                elemento_pai=elemento_referencia.find_element('xpath','..') #fico com xpath do pai
                link=elemento_pai.get_attribute('href')

                lista_ofertas.append((nome, preco, link))
                
    return lista_ofertas


def busca_buscape (nav,produto,termos_banidos,preco_min,preco_max):
    
    #tratar os valores
    produto=produto.lower()
    termos_banidos=termos_banidos.lower()
    lista_termos_nome_produto=produto.split(" ")
    lista_termos_banidos=termos_banidos.split(" ")
    lista_ofertas=[]
    preco_minimo=float(preco_min)
    preco_maximo=float(preco_max)
    
    #buscar no buscape
    nav.get("https://www.buscape.com.br/")
    nav.find_element('xpath','//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(produto,Keys.ENTER)
    
    #obter os resultados
    #time.sleep(5) # aguardar para que a página carregue 
    
    while len(nav.find_elements('class name','Select_Select__1S7HV')) < 1:
        time.sleep(1)
    
    lista_resultados=nav.find_elements('class name','SearchCard_ProductCard_Inner__7JhKb')
    
    for resultado in lista_resultados:
        preco=resultado.find_element('class name','Text_MobileHeadingS__Zxam2').text
        nome=resultado.find_element('class name','SearchCard_ProductCard_Name__ZaO5o').text 
        nome=nome.lower()
        link=resultado.get_attribute('href')
        
    #analisar se o resultado tem termo banido e tem todos os termos do produto  
        tem_termos_banidos=verificar_tem_termos_banidos(lista_termos_banidos,nome)
        tem_todos_termos_produtos=verificar_tem_termos_produtos(lista_termos_nome_produto,nome)
    
    #analisar se o preço está entre o preço mínimo e o preço máximo
        if not tem_termos_banidos and tem_todos_termos_produtos:
            preco=preco.replace('R$','').replace('.','').replace(',','.')
            preco=float(preco)
            if preco_minimo<=preco <=preco_maximo:
                lista_ofertas.append((nome,preco,link))
    #retornar resultados
    return lista_ofertas

# Construir a lista de ofertas

In [5]:
nav = webdriver.Chrome(service=servico) # codigo se instalar o navegador no ponto 2

tabela_ofertas=pd.DataFrame()

for linha in tabela_produtos.index:
    #pesquisar pelo produto google_shopping - euros
    produto=tabela_produtos.loc[linha,"Nome"]
    termos_banidos=tabela_produtos.loc[linha,"Termos banidos"]
    
    preco_min=tabela_produtos.loc[linha,"Preço mínimo"]
    preco_max=tabela_produtos.loc[linha,"Preço máximo"]
    
    preco_min_reais=tabela_produtos.loc[linha,"Preço mínimo"]*10
    preco_max_reais=tabela_produtos.loc[linha,"Preço máximo"]*10

    lista_ofertas_google_shopping = procura_gooogle_shopping(nav,produto,termos_banidos,preco_min,preco_max)
    if lista_ofertas_google_shopping:  # verifica se a lista retornada está vazia
        # como o output é uma lista de tuplas consigo transformar num dataframe
        tabela_google_shopping=pd.DataFrame(lista_ofertas_google_shopping, columns=['produtos','preco','link'])
        tabela_ofertas=pd.concat([tabela_ofertas,tabela_google_shopping])
    else:
        tabela_google_shopping=None
    
    lista_ofertas_busca_buscape = busca_buscape(nav,produto,termos_banidos,preco_min_reais,preco_max_reais)
    if lista_ofertas_busca_buscape:
        tabela_buscape=pd.DataFrame(lista_ofertas_busca_buscape, columns=['produtos','preco','link'])
        tabela_ofertas=pd.concat([tabela_ofertas,tabela_buscape])
    else:
        tabela_buscape=None

display(tabela_ofertas)

,produtos,preco,link
0,apple iphone 12 64gb preto apple,439.00,https://www.google.com/aclk?sa=l&ai=DChcSEwj9k...
1,apple iphone 12 64 gb - preto,428.00,https://www.google.com/aclk?sa=l&ai=DChcSEwj9k...
2,iphone 12 64 gb preto recondicionado - garanti...,439.99,https://www.google.com/aclk?sa=l&ai=DChcSEwj9k...
3,apple iphone 12 64gb branco apple,439.00,https://www.google.com/aclk?sa=l&ai=DChcSEwj9k...
4,apple iphone 12 64 gb - azul,422.93,https://www.google.com/aclk?sa=l&ai=DChcSEwj9k...
5,iphone 12 64 gb branco recondicionado - garant...,439.99,https://www.google.com/aclk?sa=l&ai=DChcSEwj9k...
6,apple iphone 12 64 gb - branco,420.00,https://www.google.com/aclk?sa=l&ai=DChcSEwj9k...
7,iphone 12 64 gb azul recondicionado - garantia...,439.99,https://www.google.com/aclk?sa=l&ai=DChcSEwj9k...
8,apple iphone 12 64gb vermelho apple,439.00,https://www.google.com/aclk?sa=l&ai=DChcSEwj9k...
9,apple iphone 12 64gb roxo apple,439.00,https://www.google.com/aclk?sa=l&ai=DChcSEwj9k...


# Exportar para excel

In [6]:
tabela_ofertas.to_excel("Ofertas.xlsx", index=False)

# Enviar o E-mail

In [7]:
# importar bibliotecas
import win32com.client as win32 

if len(tabela_ofertas)>0:
    
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0) 
    mail.To = 'xxxxxx@hotmail.com'
    mail.Subject = f"Produto(s) encontrados na faixa de preços desejada"
    
    mail.HTMLBody = f"""
    <p>Prezados, </p>
    <p>Encontramos alguns produtos em oferta dentro da faixa de preços sesejada </p>
    {tabela_ofertas.to_html(index=False)}
    <p>Obrigado,</p>
    <p>Daniel,</p>
    """

    mail.Display(True) # -> apenas para ver o e-mail

    #mail.Send() ->enviar o e-mail (bloqueado)



In [8]:
nav.quit() # fechar o navegador